# U-Net Model for RV Segmentation

## Training U-Net CNN

In [1]:
import sys
sys.path.append("..")
import os, re, glob
import numpy as np
import matplotlib as plt
%matplotlib inline

from src import data,unet

/Users/aksharkumar/miniconda2/envs/rvenv/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras import utils

#train_dir = "/home/ubuntu/training/TrainingSet"
train_dir = "/Users/aksharkumar/Documents/mlndCapstone/trainingdata/TrainingSet"

images=[]
inner_masks=[]
outer_masks = []

patient_directories = sorted(glob.glob(os.path.join(train_dir, "patient*")))

for patient_dir in patient_directories:
    imgdata = data.ImageData(patient_dir)
    images += imgdata.labeled_images
    inner_masks += imgdata.endo_masks.values()
    outer_masks += imgdata.epi_masks.values()

images = np.asarray(images)[:,:,:,None].astype('float64')
i_masks = np.asarray(inner_masks)
o_masks = np.asarray(outer_masks)

dims = i_masks.shape
classes = len(set(i_masks[0].flatten()))
new_shape = dims + (classes,)
i_masks = utils.to_categorical(i_masks).reshape(new_shape)
o_masks = utils.to_categorical(o_masks).reshape(new_shape)

print("There are %d total training images." % len(images))
print("There are %d total inner masks." % len(inner_masks))
print("There are %d total outer masks." % len(outer_masks))


There are 243 total training images.
There are 243 total inner masks.
There are 243 total outer masks.


In [4]:
height,width,_ = images[0].shape
dropout = 0.5

unet_conv = unet.UNet()

model = unet_conv.get_unet(height=height,width=width,channels=1,features=32,steps=3,dropout=dropout,padding='same')

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 216, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 216, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 216, 256, 32) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 216, 256, 32) 0           activation_1[0][0]               
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
from keras.optimizers import Adam
from keras import backend as K

def dice_coef(y_true, y_pred):
        flat_y_true = K.flatten(y_true)
        flat_y_preds = K.flatten(y_pred)
        intersection = K.sum(flat_y_true*flat_y_preds)
        return (2. * intersection + 1.) / (K.sum(flat_y_true)+K.sum(flat_y_preds))

def show_plots(history):    
    plt.plot(history.history['dice_coef'])
    plt.plot(history.history['val_dice_coef'])
    plt.title('model dice')
    plt.ylabel('dice')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

model.compile(optimizer=Adam(lr=1e-5),loss='categorical_crossentropy',metrics=[dice_coef])

In [6]:
#TODO: 1. split data into training and validation set
#      2. Augment the data
#      3. Train model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

from math import ceil

seed = 0

validation_split=0.2

epochs = 200
batch_size = 20


split_index = int((1 - validation_split) * len(images))

train_steps = ceil(split_index / batch_size)
val_steps = ceil((len(images)-split_index )/batch_size)

train_images = images[:split_index]
train_inner_masks = i_masks[:split_index]

validation_images = images[split_index:]
validation_inner_masks = i_masks[split_index:]


data_gen_args = dict(rotation_range=180,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     shear_range=0.1,
                     zoom_range=0.01,
                     fill_mode='nearest')

train_images_datagen = ImageDataGenerator(**data_gen_args)
train_masks_datagen = ImageDataGenerator(**data_gen_args)


seed = 1
#height,width,channels = train_images[0].shape
#print(train_images[0].shape)
#print(train_inner_masks[0].shape)
train_images_datagen.fit(train_images,augment=True,seed=seed)
train_masks_datagen.fit(train_inner_masks,augment=True,seed=seed)

train_images_generator = train_images_datagen.flow(train_images, y=None, seed=seed)
train_masks_generator = train_images_datagen.flow(train_inner_masks, y=None, seed=seed)


train_generator = zip(train_images_generator, train_masks_generator)


/Users/aksharkumar/miniconda2/envs/rvenv/lib/python3.6/site-packages/keras/preprocessing/image.py:722: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')
/Users/aksharkumar/miniconda2/envs/rvenv/lib/python3.6/site-packages/keras/preprocessing/image.py:896: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


In [14]:
checkpointer = ModelCheckpoint(filepath='saved_models/endo_models/weightsDrop50.hdf5', verbose=1, save_best_only=True)

hist = model.fit_generator(train_generator,steps_per_epoch=train_steps,epochs=epochs,verbose=1,callbacks=[checkpointer],
                   validation_data=(validation_images,validation_inner_masks),validation_steps=val_steps)
show_plots(hist)

Epoch 1/200
10/10 [==============================] - 18s - loss: 3.7656 - dice_coef: 0.6384 - val_loss: 0.6841 - val_dice_coef: 0.5046
Epoch 2/200
10/10 [==============================] - 17s - loss: 3.1180 - dice_coef: 0.6532 - val_loss: 0.6863 - val_dice_coef: 0.5035
Epoch 3/200
10/10 [==============================] - 16s - loss: 2.5540 - dice_coef: 0.6644 - val_loss: 0.6877 - val_dice_coef: 0.5027
Epoch 4/200
10/10 [==============================] - 17s - loss: 1.9731 - dice_coef: 0.6717 - val_loss: 0.6886 - val_dice_coef: 0.5023
Epoch 5/200
10/10 [==============================] - 16s - loss: 1.5972 - dice_coef: 0.6785 - val_loss: 0.6890 - val_dice_coef: 0.5021
Epoch 6/200
10/10 [==============================] - 17s - loss: 1.2416 - dice_coef: 0.6794 - val_loss: 0.6890 - val_dice_coef: 0.5021
Epoch 7/200
10/10 [==============================] - 16s - loss: 1.0452 - dice_coef: 0.6772 - val_loss: 0.6887 - val_dice_coef: 0.5022
Epoch 8/200
10/10 [==============================] - 18

10/10 [==============================] - 18s - loss: 0.5216 - dice_coef: 0.6638 - val_loss: 0.6705 - val_dice_coef: 0.5115
Epoch 30/200
10/10 [==============================] - 18s - loss: 0.5201 - dice_coef: 0.6615 - val_loss: 0.6695 - val_dice_coef: 0.5120
Epoch 31/200
10/10 [==============================] - 16s - loss: 0.5232 - dice_coef: 0.6662 - val_loss: 0.6684 - val_dice_coef: 0.5126
Epoch 32/200
10/10 [==============================] - 18s - loss: 0.5060 - dice_coef: 0.6684 - val_loss: 0.6673 - val_dice_coef: 0.5131
Epoch 33/200
10/10 [==============================] - 16s - loss: 0.5050 - dice_coef: 0.6655 - val_loss: 0.6661 - val_dice_coef: 0.5137
Epoch 34/200
10/10 [==============================] - 18s - loss: 0.5013 - dice_coef: 0.6727 - val_loss: 0.6649 - val_dice_coef: 0.5144
Epoch 35/200
10/10 [==============================] - 16s - loss: 0.4962 - dice_coef: 0.6775 - val_loss: 0.6637 - val_dice_coef: 0.5150
Epoch 36/200
10/10 [==============================] - 18s - l

10/10 [==============================] - 18s - loss: 0.4301 - dice_coef: 0.7352 - val_loss: 0.6339 - val_dice_coef: 0.5307
Epoch 56/200
10/10 [==============================] - 16s - loss: 0.4337 - dice_coef: 0.7400 - val_loss: 0.6321 - val_dice_coef: 0.5317
Epoch 57/200
10/10 [==============================] - 18s - loss: 0.4176 - dice_coef: 0.7436 - val_loss: 0.6303 - val_dice_coef: 0.5326
Epoch 58/200
10/10 [==============================] - 18s - loss: 0.4207 - dice_coef: 0.7444 - val_loss: 0.6285 - val_dice_coef: 0.5336
Epoch 59/200
10/10 [==============================] - 16s - loss: 0.4123 - dice_coef: 0.7459 - val_loss: 0.6265 - val_dice_coef: 0.5347
Epoch 60/200
10/10 [==============================] - 18s - loss: 0.4220 - dice_coef: 0.7466 - val_loss: 0.6246 - val_dice_coef: 0.5358
Epoch 61/200
10/10 [==============================] - 16s - loss: 0.4150 - dice_coef: 0.7531 - val_loss: 0.6226 - val_dice_coef: 0.5368
Epoch 62/200
10/10 [==============================] - 18s - l

10/10 [==============================] - 18s - loss: 0.3597 - dice_coef: 0.7816 - val_loss: 0.5763 - val_dice_coef: 0.5628
Epoch 82/200
10/10 [==============================] - 16s - loss: 0.3607 - dice_coef: 0.7808 - val_loss: 0.5733 - val_dice_coef: 0.5646
Epoch 83/200
10/10 [==============================] - 18s - loss: 0.3657 - dice_coef: 0.7858 - val_loss: 0.5700 - val_dice_coef: 0.5665
Epoch 84/200
10/10 [==============================] - 16s - loss: 0.3387 - dice_coef: 0.7964 - val_loss: 0.5668 - val_dice_coef: 0.5683
Epoch 85/200
10/10 [==============================] - 18s - loss: 0.3561 - dice_coef: 0.7919 - val_loss: 0.5636 - val_dice_coef: 0.5702
Epoch 86/200
10/10 [==============================] - 18s - loss: 0.3480 - dice_coef: 0.7965 - val_loss: 0.5607 - val_dice_coef: 0.5719
Epoch 87/200
10/10 [==============================] - 16s - loss: 0.3532 - dice_coef: 0.7956 - val_loss: 0.5579 - val_dice_coef: 0.5736
Epoch 88/200
10/10 [==============================] - 18s - l

10/10 [==============================] - 18s - loss: 0.3094 - dice_coef: 0.8202 - val_loss: 0.4880 - val_dice_coef: 0.6170
Epoch 108/200
10/10 [==============================] - 16s - loss: 0.2944 - dice_coef: 0.8231 - val_loss: 0.4832 - val_dice_coef: 0.6201
Epoch 109/200
10/10 [==============================] - 18s - loss: 0.3010 - dice_coef: 0.8252 - val_loss: 0.4785 - val_dice_coef: 0.6232
Epoch 110/200
10/10 [==============================] - 16s - loss: 0.3041 - dice_coef: 0.8237 - val_loss: 0.4740 - val_dice_coef: 0.6262
Epoch 111/200
10/10 [==============================] - 18s - loss: 0.3020 - dice_coef: 0.8255 - val_loss: 0.4705 - val_dice_coef: 0.6285
Epoch 112/200
10/10 [==============================] - 16s - loss: 0.2885 - dice_coef: 0.8270 - val_loss: 0.4665 - val_dice_coef: 0.6312
Epoch 113/200
10/10 [==============================] - 18s - loss: 0.3100 - dice_coef: 0.8259 - val_loss: 0.4620 - val_dice_coef: 0.6342
Epoch 114/200
10/10 [==============================] - 

10/10 [==============================] - 16s - loss: 0.2632 - dice_coef: 0.8407 - val_loss: 0.3686 - val_dice_coef: 0.7021
Epoch 134/200
10/10 [==============================] - 18s - loss: 0.2824 - dice_coef: 0.8378 - val_loss: 0.3638 - val_dice_coef: 0.7059
Epoch 135/200
10/10 [==============================] - 18s - loss: 0.2569 - dice_coef: 0.8467 - val_loss: 0.3579 - val_dice_coef: 0.7106
Epoch 136/200
10/10 [==============================] - 16s - loss: 0.2609 - dice_coef: 0.8464 - val_loss: 0.3511 - val_dice_coef: 0.7160
Epoch 137/200
10/10 [==============================] - 18s - loss: 0.2574 - dice_coef: 0.8490 - val_loss: 0.3441 - val_dice_coef: 0.7217
Epoch 138/200
10/10 [==============================] - 16s - loss: 0.2499 - dice_coef: 0.8538 - val_loss: 0.3376 - val_dice_coef: 0.7271
Epoch 139/200
10/10 [==============================] - 18s - loss: 0.2609 - dice_coef: 0.8520 - val_loss: 0.3319 - val_dice_coef: 0.7319
Epoch 140/200
10/10 [==============================] - 

10/10 [==============================] - 16s - loss: 0.2413 - dice_coef: 0.8677 - val_loss: 0.2483 - val_dice_coef: 0.8102
Epoch 160/200
10/10 [==============================] - 18s - loss: 0.2457 - dice_coef: 0.8655 - val_loss: 0.2474 - val_dice_coef: 0.8112
Epoch 161/200
10/10 [==============================] - 16s - loss: 0.2329 - dice_coef: 0.8684 - val_loss: 0.2448 - val_dice_coef: 0.8139
Epoch 162/200
10/10 [==============================] - 18s - loss: 0.2309 - dice_coef: 0.8704 - val_loss: 0.2422 - val_dice_coef: 0.8168
Epoch 163/200
10/10 [==============================] - 18s - loss: 0.2306 - dice_coef: 0.8724 - val_loss: 0.2390 - val_dice_coef: 0.8203
Epoch 164/200
10/10 [==============================] - 16s - loss: 0.2447 - dice_coef: 0.8681 - val_loss: 0.2382 - val_dice_coef: 0.8212
Epoch 165/200
10/10 [==============================] - 18s - loss: 0.2330 - dice_coef: 0.8720 - val_loss: 0.2370 - val_dice_coef: 0.8226
Epoch 166/200
10/10 [==============================] - 

In [15]:
print(np.max(hist.history['val_dice_coef']))
print(np.max(hist.history['dice_coef']))

# model.load_weights('saved_models/weights.hdf5')



0.841971736781
0.878794970879


# Results of Endocardium Model

## Training
hyperparams: {Dropout:0.5, Epochs:50, Batch Size: 20}

- train_dice = 0.62365097794
- val_dice = 0.522672422078

hyperparams: {Dropout:0.5, Epochs: 200, Batch Size: 20}

- train_dice = 0.878794970879
- val_dice = 0.841971736781


In [5]:
model_epi = unet_conv.get_unet(height=height,width=width,channels=1,features=32,steps=3,dropout=dropout,padding='same')

model_epi.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 216, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 216, 256, 32) 320         input_2[0][0]                    
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 216, 256, 32) 0           conv2d_16[0][0]                  
__________________________________________________________________________________________________
dropout_15 (Dropout)            (None, 216, 256, 32) 0           activation_15[0][0]              
__________________________________________________________________________________________________
conv2d_17 

In [7]:
seed_epi = 10

train_outer_masks = o_masks[:split_index]

train_images_datagen.fit(train_images,augment=True,seed=seed)
train_masks_datagen.fit(train_outer_masks,augment=True,seed=seed)

train_images_generator = train_images_datagen.flow(train_images, y=None, seed=seed)
train_masks_generator = train_images_datagen.flow(train_outer_masks, y=None, seed=seed)


train_outer_generator = zip(train_images_generator, train_masks_generator)


/Users/aksharkumar/miniconda2/envs/rvenv/lib/python3.6/site-packages/keras/preprocessing/image.py:722: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')
/Users/aksharkumar/miniconda2/envs/rvenv/lib/python3.6/site-packages/keras/preprocessing/image.py:896: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (194, 216, 256, 2) (2 channels).
  ' (' + str(self.x.shape[channels_axis]) + ' channels).')


In [ ]:
checkpointer = ModelCheckpoint(filepath='saved_models/epi_models/weightsNoDrop.hdf5',verbose=1,save_best_only=True)